# Brief

An earthquake catalogue is an archive of all of the earthquakes we know about, but how do we know whether we have recorded everything that has happened?

Let's take a look at some idealised examples

In [ ]:
# Just a few libraries needed this time
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, Normalize

# Visualising Catalogue Information 

The starting point in understanding completeness is simply to look at the changes in numbers of earthquakes with time.

In [ ]:
def plot_magnitude_time_density(magnitudes, years, mbins, time_bins,
                                completeness_table=[], vmin=1, vmax=100,
                                filename=None, filetype="png", dpi=300):
    """
    Create a magnitude density plot
    :param magnitudes:
        Vector of magnitudes
    :param years:
        Vector of years
    :param mbins:
        Edges of the magnitude bins
    :param time_bins:
        Edges of the time bins
    :param completeness_table:
        If present, the table of completeness
    """
    # Generate a 2-D historgram in terms of magnitude and time
    counter = np.histogram2d(years, magnitudes, bins=(time_bins,mbins))[0]
    # Plot the density
    plt.figure(figsize=(10,6))
    plt.pcolormesh(time_bins[:-1], mbins[:-1],
                   counter.T, norm=LogNorm(vmin, vmax))
    # Add axes and labels
    plt.xlabel("Year", fontsize=16)
    plt.ylabel("Magnitude", fontsize=16)
    plt.tick_params(labelsize=14)
    plt.colorbar()
    plt.grid()
    # If a completeness table is given add on a step line
    if len(completeness_table):
        completeness = np.array(completeness_table)
        plt.step(completeness[:, 0], completeness[:, 1], "k--", lw=2)
    # If the filename is specified then export to file 
    if filename:
        plt.savefig(filename, format=filetype, dpi=300, bbox_inches="tight")

### Load in the catalogue

In [ ]:
# Load in the catalogues
catalogue1 = np.genfromtxt("./test_completeness_catalogue_1.csv",
                           delimiter=",",
                           skip_header=1)
# First column is magnitudes
magnitudes1 = catalogue1[:, 0]

# Second column is years
years1 = catalogue1[:, 1]
plt.plot(years1, magnitudes1, ".")

well ... something's going on, but the plot is not so informative. Let's look at the density instead ...

### Visualise the density of events with time

In [ ]:
magnitude_bins = np.arange(3.0, 7.1, 0.1)
year_bins = np.arange(1900., 2016., 1)
plot_magnitude_time_density(magnitudes1, years1,
                            mbins=magnitude_bins,
                            time_bins=year_bins)

# Looking at changes in rates 

A surprisingly effective way of analysing the completeness is by looking at the cumulative number of events recorded in each magnitude bin 

In [ ]:
def cumulative_rates(magnitudes, years, completeness_table,
                      filename=None, filetype="png", dpi=300):
    """
    Produces a plot of the cumulative number of events in
    each magnitude bin to identify completeness
    """
    # Completeness table is a list, make it an array
    ctable = np.array(completeness_table)
    bins = ctable[:, 1]
    plt.figure(figsize=(8,8))
    for i in range(len(bins) - 1):
        #  Create a boolean vector of events within a given
        #  magnitude range - then convert to floats
        idx = np.logical_and(magnitudes >= bins[i],
                             magnitudes < bins[i + 1]).astype(float)
        if not np.any(idx):
            # If no earthquakes, skip ahead
            continue
        # Generate the cumulative number of events
        count = np.cumsum(idx)
        # Normalise so that the cumulative number of events per
        # bin goes to 1
        count /= float(count[-1])
        # Plot the cumulative number of countes
        plt.plot(years, count, "-", lw=2,
                 label="%.1f - %.1f" % (bins[i], bins[i + 1]))
        # Locate where to place the completeness marker in the
        # normalised curves
        iloc = np.searchsorted(years, completeness_table[i][0])
        line = plt.plot(years[iloc], count[iloc], "s",
                        markeredgecolor="k", markerfacecolor="r")[0]
        line.set_clip_on(False)
    #  Complete the figure setup
    plt.grid()
    plt.legend(loc=2, fontsize=14)
    plt.xlabel("Year", fontsize=16)
    plt.ylabel("Normalised Cumulative Rate", fontsize=16)
    plt.tick_params(labelsize=14)
    if filename:
        plt.savefig(filename, format=filetype, dpi=dpi, bbox_inches="tight")


Let's look at where the cumulative number of rates are changing - where do these changes occur?

In [ ]:
# Setup a "dummy" completeness table - the second column delineates the magnitude bin edges
# - we don't know the completeness time periods yet 
dummy_completeness_table = [[1900, 3.0],
                            [1900, 3.5],
                            [1900, 4.0],
                            [1900, 4.5],
                            [1900, 5.0],
                            [1900, 5.5],
                            [1900, 6.0],
                            [1900, 6.5]]

cumulative_rates(magnitudes1, years1, dummy_completeness_table)

In [ ]:
completeness_table = [[2005, 3.0],
                      [2005, 3.5],
                      [1990, 4.0],
                      [1975, 4.5],
                      [1960, 5.0],
                      [1950, 5.5],
                      [1900, 6.0],
                      [1900, 6.5]]
cumulative_rates(magnitudes1, years1, completeness_table)

### Now we can look at the density plot again

In [ ]:
magnitude_bins = np.arange(3.0, 7.1, 0.1)
year_bins = np.arange(1900., 2016., 1)
plot_magnitude_time_density(magnitudes1, years1,
                            mbins=magnitude_bins,
                            time_bins=year_bins,
                            completeness_table=completeness_table)

### This example is a little oversimplified. Now try something a bit more realistic

In [ ]:
# Load in the catalogues
catalogue2 = np.genfromtxt("./test_completeness_catalogue_2.csv",
                           delimiter=",",
                           skip_header=1)
# First column is magnitudes
magnitudes2 = catalogue2[:, 0]

# Second column is years
years2 = catalogue2[:, 1]

magnitude_bins = np.arange(3.0, 7.1, 0.1)
year_bins = np.arange(1900., 2016., 1)
plot_magnitude_time_density(magnitudes2, years2,
                            mbins=magnitude_bins,
                            time_bins=year_bins)

In [ ]:
dummy_completeness_table = [[1900, 3.0],
                            [1900, 3.5],
                            [1900, 4.0],
                            [1900, 4.5],
                            [1900, 5.0],
                            [1900, 5.5],
                            [1900, 6.0],
                            [1900, 6.5]]

cumulative_rates(magnitudes2, years2, dummy_completeness_table)

In [ ]:
completeness_table = [[2005, 3.0],
                      [2005, 3.5],
                      [1990, 4.0],
                      [1975, 4.5],
                      [1960, 5.0],
                      [1950, 5.5],
                      [1900, 6.0],
                      [1900, 6.5]]
cumulative_rates(magnitudes2, years2, completeness_table)